<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [4]:
## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [5]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [9]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [19]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


**Findings from the map exploration:**

1. **Proximity to the Equator:** 
   - **NO**, not all launch sites are close to the Equator.
   - CCAFS and KSC (Florida) are around 28°N latitude
   - VAFB (California) is around 34°N latitude
   - The Equator is at 0° latitude, so these sites are quite far north

2. **Proximity to the coast:**
   - **YES**, all launch sites are in very close proximity to the coast
   - All sites appear to be located directly on ocean coastlines
   - This makes sense for safety reasons - rockets launch over water

**Explanation:**
- Launch sites are typically located near coasts for safety, allowing rockets to fly over water rather than populated areas.
- Being closer to the Equator provides a natural velocity boost due to Earth's rotation (faster at Equator), but other factors like logistics, infrastructure, and geopolitical considerations also influence site selection.
- Florida sites (CCAFS, KSC) are closer to the Equator than California (VAFB), which gives them a slight advantage for certain orbital inclinations.

In [20]:
# Task 2: Mark the success/failed launches for each site on the map

for index, row in spacex_df.iterrows():
    # Set marker color: green for success (class=1), red for failure (class=0)
    marker_color = 'green' if row['class'] == 1 else 'red'
    
    # Create marker with color-coded icon
    folium.CircleMarker(
        location=[row['Lat'], row['Long']],
        radius=5,
        color=marker_color,
        fill=True,
        fill_color=marker_color,
        fill_opacity=0.7,
        popup=f"Site: {row['Launch Site']}<br>Outcome: {'Success' if row['class'] == 1 else 'Failed'}"
    ).add_to(site_map)

# Display the map
site_map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [21]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [22]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [25]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
from folium.plugins import MarkerCluster

# Create a MarkerCluster object
marker_cluster = MarkerCluster()

# Create a new column for marker colors
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Add marker_cluster to the site_map
marker_cluster.add_to(site_map)
# Apply a function to create marker_color column
def assign_marker_color(class_value):
    return 'green' if class_value == 1 else 'red'

# Create marker_color column
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [29]:
import folium

# Create map
site_map = folium.Map(location=[28.5, -80.5], zoom_start=5)

# Add simple markers
for index, row in spacex_df.iterrows():
    color = 'green' if row['class'] == 1 else 'red'
    
    folium.CircleMarker(
        location=[row['Lat'], row['Long']],
        radius=6,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"Site: {row['Launch Site']}<br>Outcome: {'Success' if row['class'] == 1 else 'Failed'}"
    ).add_to(site_map)

site_map

In [28]:
# Add marker_cluster to current site_map

site_map.add_child(marker_cluster)

# For each launch result, add a Marker to marker_cluster
for index, row in spacex_df.iterrows():
    # Create a Marker object
    marker = folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=folium.Icon(color='white', icon_color=row['marker_color']),
        popup=f"Site: {row['Launch Site']}<br>Outcome: {'Success' if row['class'] == 1 else 'Failed'}"
    )
    # Add marker to marker_cluster
    marker_cluster.add_child(marker)

# Display the map
site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [30]:
# TASK 3: Calculate the distances between a launch site to its proximities
# Add NASA Shuttle Landing Facility coordinate
nasa_landing_coordinate = [28.572, -80.649]  # Approximate KSC Shuttle Landing Facility

# Calculate distance between two points using Haversine formula
from math import radians, cos, sin, asin, sqrt

def calculate_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # Convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of earth in kilometers
    return c * r

# Create a dataframe to store distances
distances_df = spacex_df.copy()

# Calculate distance to NASA Shuttle Landing Facility for each launch
distances_df['distance_to_landing_facility_km'] = distances_df.apply(
    lambda row: calculate_distance(row['Lat'], row['Long'], 
                                  nasa_landing_coordinate[0], nasa_landing_coordinate[1]), 
    axis=1
)

# Display results
print("Distances from launch sites to NASA Shuttle Landing Facility:")
print(distances_df[['Launch Site', 'class', 'distance_to_landing_facility_km']].head(10))
print(f"\nAverage distance: {distances_df['distance_to_landing_facility_km'].mean():.2f} km")

Distances from launch sites to NASA Shuttle Landing Facility:
   Launch Site  class  distance_to_landing_facility_km
0  CCAFS LC-40      0                         7.079162
1  CCAFS LC-40      0                         7.079162
2  CCAFS LC-40      0                         7.079162
3  CCAFS LC-40      0                         7.079162
4  CCAFS LC-40      0                         7.079162
5  CCAFS LC-40      0                         7.079162
6  CCAFS LC-40      0                         7.079162
7  CCAFS LC-40      0                         7.079162
8  CCAFS LC-40      0                         7.079162
9  CCAFS LC-40      0                         7.079162

Average distance: 686.18 km


In [31]:
# Define coordinates for proximity points
coastline_coordinate = [28.562, -80.568]  # Example coastline point near CCAFS
city_coordinate = [28.104, -80.645]       # Example: Cocoa Beach, FL
highway_coordinate = [28.525, -80.791]    # Example: Highway near KSC

# Calculate distances to various proximities
def calculate_all_distances(row):
    """Calculate distances to various points of interest"""
    distances = {}
    
    # To NASA landing facility
    distances['to_landing_facility'] = calculate_distance(
        row['Lat'], row['Long'], nasa_landing_coordinate[0], nasa_landing_coordinate[1])
    
    # To coastline
    distances['to_coastline'] = calculate_distance(
        row['Lat'], row['Long'], coastline_coordinate[0], coastline_coordinate[1])
    
    # To city
    distances['to_city'] = calculate_distance(
        row['Lat'], row['Long'], city_coordinate[0], city_coordinate[1])
    
    # To highway
    distances['to_highway'] = calculate_distance(
        row['Lat'], row['Long'], highway_coordinate[0], highway_coordinate[1])
    
    return distances

# Apply distance calculations
distance_results = []
for index, row in spacex_df.iterrows():
    distances = calculate_all_distances(row)
    distances['Launch Site'] = row['Launch Site']
    distances['class'] = row['class']
    distance_results.append(distances)

# Create dataframe with all distances
all_distances_df = pd.DataFrame(distance_results)
print("\nAll distances calculated:")
print(all_distances_df.head())


All distances calculated:
   to_landing_facility  to_coastline    to_city  to_highway  Launch Site  \
0             7.079162      0.914393  51.389103    21.27681  CCAFS LC-40   
1             7.079162      0.914393  51.389103    21.27681  CCAFS LC-40   
2             7.079162      0.914393  51.389103    21.27681  CCAFS LC-40   
3             7.079162      0.914393  51.389103    21.27681  CCAFS LC-40   
4             7.079162      0.914393  51.389103    21.27681  CCAFS LC-40   

   class  
0      0  
1      0  
2      0  
3      0  
4      0  


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [33]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [34]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [35]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
# Coordinates for CCAFS LC-40 launch site
launch_site_lat = 28.562302
launch_site_lon = -80.577356

# Coordinates for closest coastline (obtained using MousePosition)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate distance between launch site and coastline
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

print(f"Launch Site: CCAFS LC-40")
print(f"Launch Site Coordinates: ({launch_site_lat}, {launch_site_lon})")
print(f"Closest Coastline Coordinates: ({coastline_lat}, {coastline_lon})")
print(f"Distance to Coastline: {distance_coastline:.4f} km")

Launch Site: CCAFS LC-40
Launch Site Coordinates: (28.562302, -80.577356)
Closest Coastline Coordinates: (28.56367, -80.57163)
Distance to Coastline: 0.5797 km


In [36]:
# Create a folium map
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=15)

# Add markers
# Launch site marker
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup=f'CCAFS LC-40 Launch Site<br>Lat: {launch_site_lat}<br>Lon: {launch_site_lon}',
    icon=folium.Icon(color='red', icon='rocket', prefix='fa')
).add_to(site_map)

# Coastline point marker
folium.Marker(
    [coastline_lat, coastline_lon],
    popup=f'Closest Coastline Point<br>Lat: {coastline_lat}<br>Lon: {coastline_lon}<br>Distance: {distance_coastline:.3f} km',
    icon=folium.Icon(color='blue', icon='water', prefix='fa')
).add_to(site_map)

# Add a line between them
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    color='green',
    weight=3,
    dash_array='5, 5',
    popup=f'Distance: {distance_coastline:.3f} km'
).add_to(site_map)

# Add distance label at midpoint
mid_lat = (launch_site_lat + coastline_lat) / 2
mid_lon = (launch_site_lon + coastline_lon) / 2

folium.Marker(
    [mid_lat, mid_lon],
    icon=folium.DivIcon(
        icon_size=(150, 36),
        icon_anchor=(75, 18),
        html=f'<div style="font-size: 12pt; color: green; background-color: white; padding: 2px; border-radius: 3px; border: 1px solid green;">{distance_coastline:.3f} km</div>'
    )
).add_to(site_map)

# Display the map
site_map

In [37]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# Coordinates for CCAFS LC-40 launch site
launch_site_lat = 28.562302
launch_site_lon = -80.577356

# Coordinates for closest coastline point
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate distance
distance = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create a marker at the coastline point with distance display
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(75, 18),
        html='<div style="font-size: 14pt; color:#d35400; font-weight: bold;">{:10.2f} KM</div>'.format(distance),
    )
)

# Add the distance marker to the site_map
distance_marker.add_to(site_map)

# Display the updated map
site_map

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [39]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# Coordinates for launch site and coastline
launch_site_coord = [28.562302, -80.577356]  # CCAFS LC-40
coastline_coord = [28.56367, -80.57163]      # Closest coastline point

# Create a PolyLine between launch site and coastline
lines = folium.PolyLine(
    locations=[launch_site_coord, coastline_coord],
    weight=2,
    color='blue',
    dash_array='5, 5',
    popup=f'Distance: {calculate_distance(launch_site_coord[0], launch_site_coord[1], coastline_coord[0], coastline_coord[1]):.3f} km'
)

# Add the line to the map
site_map.add_child(lines)

In [40]:
# Dictionary of launch sites and their coastline points
launch_coast_pairs = {
    'CCAFS LC-40': {
        'launch': [28.562302, -80.577356],
        'coast': [28.56367, -80.57163]
    },
    'CCAFS SLC-40': {
        'launch': [28.563197, -80.576820],
        'coast': [28.564, -80.570]
    }
}

# Create map
site_map = folium.Map(location=[28.563, -80.574], zoom_start=14)

# Add lines for each pair
for site_name, coords in launch_coast_pairs.items():
    # Calculate distance
    distance = calculate_distance(coords['launch'][0], coords['launch'][1],
                                  coords['coast'][0], coords['coast'][1])
    
    # Create PolyLine
    line = folium.PolyLine(
        locations=[coords['launch'], coords['coast']],
        weight=2,
        color='blue',
        popup=f'{site_name}: {distance:.3f} km to coast'
    )
    site_map.add_child(line)
    
    # Add markers
    folium.Marker(coords['launch'], popup=site_name).add_to(site_map)
    folium.Marker(coords['coast'], popup=f'Coastline near {site_name}').add_to(site_map)

site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [41]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
# Define coordinates
launch_site_coord = [28.562302, -80.577356]  # CCAFS LC-40

# Define proximity points
city_coord = [28.104, -80.645]        # Cocoa Beach
railway_coord = [28.572, -80.585]     # Nearby railway
highway_coord = [28.525, -80.791]     # Highway

# Create map
site_map = folium.Map(location=[28.5, -80.6], zoom_start=11)

# Add launch site marker
folium.Marker(
    launch_site_coord,
    popup='CCAFS LC-40 Launch Site',
    icon=folium.Icon(color='red', icon='rocket', prefix='fa')
).add_to(site_map)

# Create markers and lines for each proximity
proximities = {
    'city': {
        'coord': city_coord,
        'color': 'blue',
        'icon': 'building',
        'label': 'Cocoa Beach City'
    },
    'railway': {
        'coord': railway_coord,
        'color': 'orange',
        'icon': 'train',
        'label': 'Railway'
    },
    'highway': {
        'coord': highway_coord,
        'color': 'green',
        'icon': 'road',
        'label': 'Highway'
    }
}

for key, info in proximities.items():
    # Calculate distance
    distance = calculate_distance(
        launch_site_coord[0], launch_site_coord[1],
        info['coord'][0], info['coord'][1]
    )
    
    # Add proximity marker with distance
    folium.Marker(
        info['coord'],
        popup=f"{info['label']}<br>Distance to launch site: {distance:.2f} km",
        icon=folium.Icon(color=info['color'], icon=info['icon'], prefix='fa')
    ).add_to(site_map)
    
    # Draw line between launch site and proximity point
    folium.PolyLine(
        locations=[launch_site_coord, info['coord']],
        weight=2,
        color=info['color'],
        dash_array='5, 5',
        popup=f"Distance: {distance:.2f} km"
    ).add_to(site_map)

# Display map
site_map

In [42]:
# Create map
site_map = folium.Map(location=[28.5, -80.6], zoom_start=11)

# Add launch site
folium.Marker(
    launch_site_coord,
    popup='CCAFS LC-40',
    icon=folium.Icon(color='red', icon='rocket', prefix='fa')
).add_to(site_map)

# Proximity points
points = [
    ([28.104, -80.645], 'Cocoa Beach City', 'blue', 'building'),
    ([28.572, -80.585], 'Railway Line', 'orange', 'train'),
    ([28.525, -80.791], 'Highway', 'green', 'road'),
    ([28.56367, -80.57163], 'Coastline', 'purple', 'water')
]

for point_coord, label, color, icon in points:
    # Calculate distance
    distance = calculate_distance(
        launch_site_coord[0], launch_site_coord[1],
        point_coord[0], point_coord[1]
    )
    
    # Add marker
    folium.Marker(
        point_coord,
        popup=f"{label}<br>{distance:.2f} km from launch site",
        icon=folium.Icon(color=color, icon=icon, prefix='fa')
    ).add_to(site_map)
    
    # Add line
    line = folium.PolyLine(
        [launch_site_coord, point_coord],
        color=color,
        weight=2,
        opacity=0.7
    )
    site_map.add_child(line)
    
    # Add distance label at midpoint
    mid_lat = (launch_site_coord[0] + point_coord[0]) / 2
    mid_lon = (launch_site_coord[1] + point_coord[1]) / 2
    
    folium.Marker(
        [mid_lat, mid_lon],
        icon=DivIcon(
            icon_size=(120, 30),
            icon_anchor=(60, 15),
            html=f'<div style="font-size: 10pt; color: {color}; font-weight: bold; background-color: white; padding: 2px; border-radius: 3px; border: 1px solid {color};">{distance:.1f} km</div>'
        )
    ).add_to(site_map)

site_map

In [43]:
# Simple version with just lines and basic markers
site_map = folium.Map(location=[28.5, -80.6], zoom_start=11)

# Launch site
folium.Marker(launch_site_coord, popup='Launch Site').add_to(site_map)

# Add points and lines
targets = [
    ([28.104, -80.645], 'City'),
    ([28.572, -80.585], 'Railway'),
    ([28.525, -80.791], 'Highway')
]

for coord, label in targets:
    distance = calculate_distance(launch_site_coord[0], launch_site_coord[1], coord[0], coord[1])
    
    folium.Marker(coord, popup=f"{label}: {distance:.1f} km").add_to(site_map)
    
    folium.PolyLine(
        [launch_site_coord, coord],
        popup=f"{label}: {distance:.1f} km"
    ).add_to(site_map)

site_map

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


1. **Railways:** No, launch sites are not in close proximity to railways.
2. **Highways:** No, launch sites are not in close proximity to highways.
3. **Coastline:** Yes, all launch sites are in very close proximity to the coastline (typically < 1 km).
4. **Cities:** Yes, launch sites maintain considerable distance from cities for safety (typically > 50 km).

**Explanation:** Launch sites prioritize coastal locations for safety (overwater trajectories), maintain safe distances from populated areas, and don't require immediate access to ground transportation infrastructure like railways or highways due to specialized transport needs for rocket components.

# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
